### Testing Realization Parameters

In [24]:
import json
import fiona
import geopandas
from collections import OrderedDict

In [25]:
gpkg = 'cat-679069/cat-679069_upstream_subset.gpkg'

In [26]:
fiona.listlayers(gpkg)

['flowpaths',
 'divides',
 'nexus',
 'crosswalk',
 'flowpath_edge_list',
 'flowpath_attributes',
 'cfe_noahowp_attributes',
 'forcing_metadata']

In [4]:
flowpaths = geopandas.read_file(gpkg, layer='flowpaths')
flowpaths.dtypes

id                         object
lengthkm                  float64
main_id                     int64
member_comid               object
tot_drainage_areasqkm     float64
order                     float64
realized_catchment         object
toid                       object
geometry                 geometry
dtype: object

In [5]:
divides = geopandas.read_file(gpkg, layer='divides')
divides.dtypes

id            object
areasqkm     float64
type          object
toid          object
geometry    geometry
dtype: object

In [6]:
nexus = geopandas.read_file(gpkg, layer='nexus')
nexus.dtypes

id            object
type          object
toid          object
geometry    geometry
dtype: object

In [7]:
crosswalk = geopandas.read_file(gpkg, layer='crosswalk')
crosswalk.dtypes

id                        object
toid                      object
NHDPlusV2_COMID          float64
NHDPlusV2_COMID_part     float64
reconciled_ID            float64
mainstem                 float64
POI_ID                    object
POI_TYPE                  object
POI_VALUE                 object
geometry                geometry
dtype: object

In [8]:
flowpath_edge_list = geopandas.read_file(gpkg, layer='flowpath_edge_list')
flowpath_edge_list.dtypes

id            object
toid          object
geometry    geometry
dtype: object

In [9]:
flowpath_attributes = geopandas.read_file(gpkg, layer='flowpath_attributes')
flowpath_attributes.dtypes

id                        object
rl_gages                  object
rl_NHDWaterbodyComID      object
Qi                       float64
MusK                     float64
MusX                     float64
n                        float64
So                       float64
ChSlp                    float64
BtmWdth                  float64
time                     float64
Kchan                    float64
nCC                      float64
TopWdthCC                float64
TopWdth                  float64
length_m                 float64
geometry                geometry
dtype: object

In [10]:
cfe_noahowp_attributes = geopandas.read_file(gpkg, layer='cfe_noahowp_attributes')
cfe_noahowp_attributes.dtypes

id                             object
gw_Coeff                       object
gw_Zmax                        object
gw_Expon                       object
ISLTYP                          int64
IVGTYP                          int64
bexp_soil_layers_stag=1       float64
bexp_soil_layers_stag=2       float64
bexp_soil_layers_stag=3       float64
bexp_soil_layers_stag=4       float64
dksat_soil_layers_stag=1      float64
dksat_soil_layers_stag=2      float64
dksat_soil_layers_stag=3      float64
dksat_soil_layers_stag=4      float64
psisat_soil_layers_stag=1     float64
psisat_soil_layers_stag=2     float64
psisat_soil_layers_stag=3     float64
psisat_soil_layers_stag=4     float64
cwpvt                         float64
mfsno                         float64
mp                            float64
refkdt                        float64
slope                         float64
smcmax_soil_layers_stag=1     float64
smcmax_soil_layers_stag=2     float64
smcmax_soil_layers_stag=3     float64
smcmax_soil_

In [11]:
forcing_metadata = geopandas.read_file(gpkg, layer='forcing_metadata')
forcing_metadata.dtypes

id                object
areasqkm         float64
cetroid_lon      float64
centroid_lat     float64
elevation        float64
slope_m_km       float64
aspect           float64
geometry        geometry
dtype: object

In [12]:
# Sample Data
# ───────┬─────────────────────────────────────────────────────────────────────────────
#        │ File: awi_config.ini
# ───────┼─────────────────────────────────────────────────────────────────────────────
#    1   │ forcing_file=BMI
#    2   │ surface_partitioning_scheme=Schaake
#    3   │ soil_params.depth=2.0[m]
#    4   │ soil_params.b=8.93396282196045[]
#    5   │ soil_params.satdk=3.19069084890877e-05[m s-1]
#    6   │ soil_params.satpsi=3.98730560956446[m]
#    7   │ soil_params.slop=0.057029859113015[m/m]
#    8   │ soil_params.smcmax=0.401686143900526[m/m]
#    9   │ soil_params.wltsmc=0.048334490431746[m/m]
#   10   │ soil_params.expon=1.0[]
#   11   │ soil_params.expon_secondary=1.0[]
#   12   │ refkdt=3.72730851635058
#   13   │ max_gw_storage=0.016[m]
#   14   │ Cgw=0.0018[m h-1]
#   15   │ expon=6.0[]
#   16   │ gw_storage=0.05[m/m]
#   17   │ alpha_fc=0.33
#   18   │ soil_storage=0.05[m/m]
#   19   │ K_nash=0.03[]
#   20   │ K_lf=0.01[]
#   21   │ nash_storage=0.0,0.0
#   22   │ num_timesteps=1
#   23   │ verbosity=1
#   24   │ DEBUG=0
#   25   │ giuh_ordinates=1.00,0.00

In [27]:
# reference doc
# https://github.com/NOAA-OWP/ngen/blob/master/doc/REALIZATION_CONFIGURATION.md
catchment_configs = {}
for idx, row in cfe_noahowp_attributes.iterrows():
    d = OrderedDict()
    
    # static parameters
    d['forcing_file']='BMI'
    d['surface_partitioning_scheme']='Schaake'

    # ----------------    
    # State Parameters
    # ----------------

    # soil depth
    d['soil_params.depth']='2.0[m]'

    # many of these values are taken from the 2m depth in hydrofabrics cfe_noahowp_attributes
    d['soil_params.b']=f'{row["bexp_soil_layers_stag=2"]}[]' # 	beta exponent on Clapp-Hornberger (1978) soil water relations

    # saturated hydraulic conductivity
    d['soil_params.satdk']=f'{row["dksat_soil_layers_stag=2"]}[m s-1]' 
    
    # saturated capillary head
    d['soil_params.satpsi']=f'{row["psisat_soil_layers_stag=2"]}[m]'
    
    # this factor (0-1) modifies the gradient of the hydraulic head at the soil bottom. 0=no-flow.
    d['soil_params.slop']=f'{row["slope"]}[m/m]'
    
    # saturated soil moisture content
    d['soil_params.smcmax']=f'{row["smcmax_soil_layers_stag=2"]}[m/m]'
    
    # wilting point soil moisture content
    d['soil_params.wltsmc']=f'{row["smcwlt_soil_layers_stag=2"]}[m/m]'
    
    # ---------------------    
    # Adjustable Parameters
    # ---------------------
    
    # optional; defaults to 1.0
    d['soil_params.expon']=f'{row["gw_Expon"]}[]' if row["gw_Expon"] is not None else '1.0[]'
    
    # optional; defaults to 1.0 
    # not sure if this is the correct key
    d['soil_params.expon_secondary']=f'{row["gw_Coeff"]}[]' if row["gw_Coeff"] is not None else '1.0[]'
    
    # maximum storage in the conceptual reservoir
    d['max_gw_storage']=f'{row["gw_Zmax"]}[m]' if row["gw_Zmax"] is not None else '0.011[m]'

     # primary outlet coefficient
    d['Cgw']='0.0018[m h-1]'

    # exponent parameter (1.0 for linear reservoir)
    d['expon']='6.0[]' 

    # initial condition for groundwater reservoir - it is the ground water as a 
    # decimal fraction of the maximum groundwater storage (max_gw_storage) for the initial timestep
    d['gw_storage']='0.05[m/m]'

    # field capacity
    d['alpha_fc']='0.33' 
    
    # initial condition for soil reservoir - it is the water in the soil as a 
    # decimal fraction of maximum soil water storage (smcmax * depth) for the initial timestep
    d['soil_storage']='0.05[m/m]' 
    
    # number of Nash lf reservoirs (optional, defaults to 2, ignored if storage values present)
    d['K_nash']='0.03[]' 
    
    # Nash Config param - primary reservoir
    d['K_lf']='0.01[]' 
    
    # Nash Config param - secondary reservoir
    d['nash_storage']='0.0,0.0' 
    
    # Giuh ordinates in dt time steps
    d['giuh_ordinates']='1.00,0.00'
    
    # ---------------------    
    # Time Info
    # ---------------------
    
    
    # set to 1 if forcing_file=BMI
    d['num_timesteps']='1' 
    
    # ---------------------    
    # Options
    # ---------------------
    
    # prints various debug and bmi info
    d['verbosity']='1' 
    
    d['DEBUG']='0'
    
    # Parameter in the surface runoff parameterization (https://mikejohnson51.github.io/hyAggregate/#Routing_Attributes)
    d['refkdt']=f'{row["refkdt"]}'
    
    catchment_configs[row.id] = d
    

In [28]:
base = 'cat-679069'
for name, conf in catchment_configs.items():
    with open(f'{base}/{name}_config.ini', 'w') as f:
        for k, v in conf.items():
            f.write(f'{k}={v}\n')

In [40]:
class CatchmentRealizations:
    def __init__(self):
        self.catchments = {}
    def add_realization(self, name, realization):
        self.catchments[name] = realization
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=False, indent=4)

class Realization:
    def __init__(self, formulation, forcing={}, time={}):
        self.formulations = formulation
        self.forcing = forcing
        self.time = time
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=False, indent=4)

class Formulation:
    def __init__(self, name, params={}, modules=[]):
        self.name = name
        self.params = params
        self.params['modules'] = modules
#        self.params.extend(modules)
#        self.modules = modules
    def add_module(self, module):
        self.modules.append(module)
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=False, indent=4)
        
class Module:
    def __init__(self, name, library_file, registration_function, params={}):
        self.name = name
        self.library_file = library_file
        self.registration_function = registration_function
        self.params=params
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=False, indent=4)

def ComplexHandler(Obj):
    if hasattr(Obj, 'toJSON'):
        return Obj.toJSON()
    else:
        raise TypeError


In [41]:
catchment_realizations = CatchmentRealizations()
for key, val in catchment_configs.items():
    config_name = f'{key}_config.ini'
    
    # CFE
    module_params = {"name": "bmi_c",
                     "model_type_name": "CFE",
                     "main_output_variable": "Q_OUT",
                     "init_config": f"{config_name}",
                     "allow_exceed_end_time": True,
                     "fixed_time_step": False,
                     "uses_forcing_file": True,
                     "variables_names_map": {
                         "atmosphere_water__liquid_equivalent_precipitation_rate": "precip_rate",
                         "water_potential_evaporation_flux": "EVAPOTRANS",
                         "ice_fraction_schaake": "sloth_ice_fraction_schaake",
                         "ice_fraction_xinan": "sloth_ice_fraction_xinan",
                         "soil_moisture_profile": "sloth_smp"
                     }}
    m1 = Module('bmi_c','/dmod/shared_libs/libcfebmi.so.1.0.0','register_bmi_cfe', params=module_params)

    # SLOTH

    module_params = {"name": "bmi_c++",
                     "model_type_name": "SLOTH",
                     "main_output_variable": "z",
                     "init_config": "/dev/null",
                     "allow_exceed_end_time": True,
                     "fixed_time_step": False,
                     "uses_forcing_file": False,
                     "model_params": {
                         "sloth_ice_fraction_schaake(1,double,m,node)": "0.0",
                         "sloth_ice_fraction_xinan(1,double,1,node)": "0.0",
                         "sloth_smp(1,double,1,node)": "0.0",
                         "EVAPOTRANS": "0.0"
                     }}
    m2 = Module('bmi_c++','/dmod/shared_libs/libslothmodel.so','none', params=module_params)


    form_params = {"name": "bmi_multi",
                   "model_type_name": "NoahOWP_CFE",
                   "main_output_variable": "Q_OUT",
                   "init_config": "",
                   "allow_exceed_end_time": False,
                   "fixed_time_step": False,
                   "uses_forcing_file": False}
    f = Formulation('bmi_multi', params=form_params, modules=[m1, m2])

    realization = Realization(f,
                              forcing={'path': f'../forcing/{key}.csv'},
                              time={'start_time': '2010-01-01 00:00:00',
                                    'end_time'  : '2010-07-01 00:00:00',
                                    'output_interval': 3600})

    catchment_realizations.add_realization(key, realization)

In [42]:
print(catchment_realizations.toJSON())

{
    "catchments": {
        "cat-679889": {
            "formulations": {
                "name": "bmi_multi",
                "params": {
                    "name": "bmi_multi",
                    "model_type_name": "NoahOWP_CFE",
                    "main_output_variable": "Q_OUT",
                    "init_config": "",
                    "allow_exceed_end_time": false,
                    "fixed_time_step": false,
                    "uses_forcing_file": false,
                    "modules": [
                        {
                            "name": "bmi_c",
                            "library_file": "/dmod/shared_libs/libcfebmi.so.1.0.0",
                            "registration_function": "register_bmi_cfe",
                            "params": {
                                "name": "bmi_c",
                                "model_type_name": "CFE",
                                "main_output_variable": "Q_OUT",
                                "init_config": "cat-6

In [43]:
with open(f'{base}/realization.json', 'w') as f:
    f.write(catchment_realizations.toJSON())